### PROJECT 1

Docelowy notebook w którym będzie finalny kod :-)

In [2]:
# imports
import torch.nn as nn
import torch.optim as optim
import numpy as np
from source.utils.config_manager import ConfigManager

# hyperparameters tuning
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory

# networks
from source.custom_cnn.pytorch_ray_tutorial_cnn import Net

In [3]:
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)
gpus_per_trial = 2
max_num_epochs = 10
num_samples = 10
    
config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.SGD]),
    "lr": tune.loguniform(1e-4, 1e-1),
}

config_net_1 = {
    "type": tune.choice([Net]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config_net_2 = {
    "type": tune.choice([Net]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config = {
    "data_loaders_factory": tune.choice([
        CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_train_transformer(), TestTransformersFactory.get_test_transformer())
    ]),
    "batch_size": tune.choice([2, 4, 8, 16]),
    "epochs": 10,
    "net": tune.choice([config_net_1, config_net_2])
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = JupyterNotebookReporter(max_progress_rows=20,overwrite=True,max_report_frequency  = 10,max_error_rows=5)

In [4]:
tuner = HyperparameteresTunner()

In [ ]:
tuner.tune(dataset_name, config, scheduler, reporter)